# Tech Challenge 3

### Verificação do ambiente de execução (Python, PyTorch e GPU)

Este bloco de código verifica as principais informações do ambiente de execução para garantir que o PyTorch esteja instalado corretamente e que o hardware disponível (como GPU) seja compatível com o processamento acelerado.

**Entradas**
- Nenhuma entrada direta — o código apenas consulta o ambiente atual.

**Processo (passo a passo)**
1. Importa as bibliotecas necessárias (`torch`, `platform`, `importlib`, `os`).
2. Exibe a versão do Python e do PyTorch instaladas.
3. Verifica se o PyTorch detecta uma GPU CUDA disponível.
4. Caso uma GPU seja encontrada:
   - Exibe o nome da GPU.
   - Checa se há suporte para operações em formato BF16 (bfloat16), importante para treinamento eficiente em hardware mais recente.
5. Caso nenhuma GPU seja detectada, informa que o sistema está rodando apenas em CPU.

**Saídas**
- Impressão no console das versões do Python e PyTorch.
- Status de disponibilidade da GPU CUDA.
- Nome da GPU e suporte a BF16 (se aplicável).

**Observações**
- Útil para depuração de ambiente antes de treinar modelos.
- Caso `torch.cuda.is_available()` retorne `False`, o treinamento usará CPU, o que pode reduzir significativamente a performance.
- Recomenda-se executar esse bloco logo no início do notebook para garantir compatibilidade de hardware e bibliotecas.



In [1]:
import torch, platform, importlib, os

print("Python:", platform.python_version())
print("PyTorch:", torch.__version__)
print("CUDA disponível:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    bf16_ok = bool(getattr(torch.cuda, "is_bf16_supported", lambda: False)())
    print("BF16 suportado:", bf16_ok)
else:
    print("Sem GPU CUDA detectada.")


Python: 3.11.9
PyTorch: 2.5.1+cu121
CUDA disponível: True
GPU: NVIDIA GeForce RTX 4060 Ti
BF16 suportado: True


### Definição dos caminhos, modelo base e hiperparâmetros de treinamento

Este bloco configura o ambiente de treinamento, definindo os diretórios, arquivos, modelo base e parâmetros principais do ajuste fino (fine-tuning) do modelo de linguagem.

**Entradas**
- Arquivos de dados:
  - `trn.json`: conjunto de treino.
  - `tst.json`: conjunto de teste.
- Caminhos e diretórios dentro da pasta `./data` e `./models`.

**Processo (passo a passo)**
1. Importa as bibliotecas necessárias (`pathlib`, `os`, `random`).
2. Define os caminhos absolutos dos arquivos de treino e teste.
3. Cria as pastas de saída:
   - `./models/prepared`: onde serão armazenados os dados pré-processados.
   - `./models/out-sft`: onde o modelo ajustado será salvo.
4. Escolhe um **modelo base pequeno** — `TinyLlama/TinyLlama-1.1B-Chat-v1.0` — compatível com GPUs comuns.  
   *(Alternativa: `Qwen/Qwen2.5-1.5B-Instruct`, também leve e otimizada para chat.)*
5. Define os **hiperparâmetros de treinamento**, como:
   - `num_train_epochs`: número de épocas.
   - `batch_size`: tamanho dos lotes por dispositivo.
   - `grad_acc_steps`: passos de acumulação de gradiente.
   - `learning_rate`: taxa de aprendizado.
   - `max_seq_len`: tamanho máximo de sequência.
   - Parâmetros LoRA (`lora_r`, `lora_alpha`, `lora_dropout`) para adaptação leve do modelo.
   - `seed`: semente de aleatoriedade para reprodutibilidade.
6. Inicializa o gerador aleatório (`random.seed`) com a semente definida.
7. Exibe no console o modelo selecionado, os caminhos dos arquivos de dados e os diretórios de saída.

**Saídas**
- Impressão das configurações principais no console.
- Criação (se inexistentes) das pastas `./models/prepared` e `./models/out-sft`.

**Observações**
- O modelo escolhido (`TinyLlama`) é adequado para testes e protótipos rápidos.
- Os hiperparâmetros devem ser ajustados conforme a capacidade da GPU disponível.
- Garantir que os arquivos `trn.json` e `tst.json` existam antes de prosseguir para evitar erros no pré-processamento.


In [4]:
from pathlib import Path
import os, random

# Caminhos dos arquivos que você já tem
TRAIN_PATH = Path("./data/trn.json")
TEST_PATH  = Path("./data/tst.json")

# Pastas de saída
OUT_DIR   = Path("./models/out-sft")
PREP_DIR  = Path("./models/prepared")
PREP_DIR.mkdir(parents=True, exist_ok=True)

# Modelo base pequeno para caber em GPU comum
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"   # alternativa: "Qwen/Qwen2.5-1.5B-Instruct"

# Hiperparâmetros (ajuste conforme seu hardware)
CFG = dict(
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    grad_acc_steps=16,
    learning_rate=2e-4,
    max_seq_len=1024,
    lora_r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    seed=42,
)

random.seed(CFG["seed"])

print("BASE_MODEL:", BASE_MODEL)
print("Arquivos:", TRAIN_PATH.resolve(), "|", TEST_PATH.resolve())
print("Saídas:", "OUT_DIR=", OUT_DIR, "| PREP_DIR=", PREP_DIR)


BASE_MODEL: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Arquivos: C:\Users\PC\Documents\Tech Challenge 3 (1)\data\trn.json | C:\Users\PC\Documents\Tech Challenge 3 (1)\data\tst.json
Saídas: OUT_DIR= models\out-sft | PREP_DIR= models\prepared


### Leitura e validação dos arquivos de dados em formato JSON/JSONL

Este bloco carrega os conjuntos de **treinamento** e **teste** a partir de arquivos JSON, com suporte flexível tanto para formato de lista (`.json`) quanto para formato linha a linha (`.jsonl`). Ele garante que os dados sejam lidos corretamente e exibe um exemplo do conteúdo para verificação.

**Entradas**
- Caminhos definidos anteriormente:
  - `TRAIN_PATH` → `./data/trn.json`
  - `TEST_PATH` → `./data/tst.json`

**Processo (passo a passo)**
1. Define a função `read_any_json(path)`:
   - Lê o conteúdo do arquivo indicado, forçando codificação UTF-8 e ignorando erros de leitura.
   - Primeira tentativa: interpreta o conteúdo completo como uma **lista JSON** (`[{}, {}, ...]`).
   - Se falhar, aplica um **fallback** para o formato **JSONL** (um JSON por linha):
     - Lê o arquivo linha por linha.
     - Ignora linhas vazias ou inválidas.
     - Converte cada linha válida em um dicionário e adiciona à lista de resultados.
2. Carrega os arquivos de treino e teste usando a função.
3. Exibe no console:
   - O número total de registros carregados em cada conjunto.
   - Um exemplo da primeira entrada do treino, mostrando campos típicos: `uid`, `title`, `content`, `target_ind`, `target_rel`.

**Saídas**
- Listas Python contendo os dados brutos (`train_raw` e `test_raw`).
- Impressão com a contagem e um exemplo representativo dos registros.

**Observações**
- A função é tolerante a erros e ignora linhas corrompidas, o que facilita o trabalho com bases parcialmente inconsistentes.
- Essa abordagem é útil quando não se sabe de antemão se o dataset está em formato JSON puro ou JSONL.
- O exemplo impresso serve para inspecionar a estrutura das chaves e validar o schema antes do pré-processamento.


In [ ]:
import json

def read_any_json(path: Path):
    text = path.read_text(encoding="utf-8", errors="ignore").strip()
    rows = []

    # Tenta lista JSON
    try:
        obj = json.loads(text)
        if isinstance(obj, list):
            rows = obj
            return rows
    except Exception:
        pass

    # Fallback: tenta JSONL
    rows = []
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            rows.append(json.loads(line))
        except Exception:
            # Ignora linhas quebradas
            continue
    return rows

train_raw = read_any_json(TRAIN_PATH)
test_raw  = read_any_json(TEST_PATH)

print(f"Carregado treino bruto: {len(train_raw)}")
print(f"Carregado teste bruto:  {len(test_raw)}")
if train_raw:
    ex = {k: train_raw[0].get(k) for k in ("uid","title","content","target_ind","target_rel")}
    print("Exemplo treino:", ex)


Carregado treino bruto: 2248619
Carregado teste bruto:  970237
Exemplo treino: {'uid': '0000031909', 'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length', 'target_ind': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 111], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}


### Limpeza, normalização e deduplicação dos dados de entrada

Este bloco realiza o **pré-processamento textual** dos conjuntos de treino e teste, eliminando registros inválidos, padronizando o texto e removendo duplicatas. O objetivo é garantir que os dados usados no ajuste fino sejam consistentes e livres de ruído.

**Entradas**
- `train_raw` e `test_raw`: listas de dicionários carregadas anteriormente com os dados brutos.
- Diretório de saída `PREP_DIR` (ex.: `./models/prepared`).

**Processo (passo a passo)**
1. Define um conjunto de strings consideradas nulas (`NULL_STRINGS`), incluindo variações como `"na"`, `"n/a"`, `"null"`, `"nan"`, `"-"`, etc.
2. Cria a função `norm_text(x)`:
   - Converte valores numéricos em string (ignorando `NaN`).
   - Remove espaços extras e múltiplos espaços entre palavras.
   - Retorna uma string limpa e padronizada.
3. Define a função `is_bad(s)`:
   - Verifica se o texto é nulo, vazio ou contém apenas placeholders genéricos.
4. Implementa `clean_rows(rows)`:
   - Aplica `norm_text` ao `title` e `content` de cada registro.
   - Remove registros sem conteúdo útil.
   - Deduplica os dados com base no `uid` (ou `title`, caso o `uid` esteja ausente).
5. Aplica `clean_rows` aos conjuntos de treino e teste.
6. Exibe estatísticas sobre o processo de limpeza:
   - Quantos registros foram removidos.
   - Quantos permanecem após deduplicação.
7. Salva as versões limpas dos datasets em formato **JSONL**, permitindo auditoria posterior:
   - `trn_clean.jsonl`
   - `tst_clean.jsonl`
8. Mostra um exemplo de registro limpo para inspeção visual.

**Saídas**
- Arquivos limpos gravados em `PREP_DIR`.
- Impressão das contagens de registros removidos e exemplo do resultado final.

**Observações**
- Essa limpeza previne falhas no tokenizador e evita que textos vazios prejudiquem o treinamento.
- O uso de `json.dumps(..., ensure_ascii=False)` preserva acentuação e caracteres Unicode.
- Embora o teste geralmente venha pré-processado, aplicá-la garante consistência entre os conjuntos.
- É importante realizar auditoria dos dados limpos antes de seguir para a etapa de tokenização.


In [6]:
import math
import re

NULL_STRINGS = {"", " ", "na", "n/a", "none", "null", "nan", "-", "--", "unknown"}

def norm_text(x):
    if x is None:
        return ""
    if isinstance(x, (int, float)):
        try:
            if math.isnan(x):
                return ""
        except Exception:
            pass
        return str(x)
    x = str(x).strip()
    # Remove múltiplos espaços
    x = re.sub(r"\s+", " ", x)
    return x

def is_bad(s: str):
    s_norm = s.strip().lower()
    return (s_norm in NULL_STRINGS) or (len(s_norm) == 0)

def clean_rows(rows):
    cleaned = []
    for r in rows:
        title = norm_text(r.get("title"))
        content = norm_text(r.get("content"))
        # Filtra registros nulos/inúteis
        if is_bad(title) or is_bad(content):
            continue
        cleaned.append({
            "uid": r.get("uid"),
            "title": title,
            "content": content,
            "target_ind": r.get("target_ind"),
            "target_rel": r.get("target_rel"),
        })
    # Deduplicar (prioriza UID, senão título)
    seen = set()
    dedup = []
    for r in cleaned:
        key = r["uid"] if r["uid"] else r["title"]
        if key in seen:
            continue
        seen.add(key)
        dedup.append(r)
    return dedup

train = clean_rows(train_raw)
test  = clean_rows(test_raw)   # normalmente teste já vem limpo, mas não custa
print(f"Treino após limpeza/dedup: {len(train)} (removidos: {len(train_raw)-len(train)})")
print(f"Teste após limpeza/dedup:  {len(test)}  (removidos: {len(test_raw)-len(test)})")

# Salvar versões limpas (para auditoria)
with (PREP_DIR/"trn_clean.jsonl").open("w", encoding="utf-8") as f:
    for r in train:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

with (PREP_DIR/"tst_clean.jsonl").open("w", encoding="utf-8") as f:
    for r in test:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

if train:
    print("Exemplo pós-limpeza:", {k: train[0][k] for k in ("title","content")})


Treino após limpeza/dedup: 1390102 (removidos: 858517)
Teste após limpeza/dedup:  599631  (removidos: 370606)
Exemplo pós-limpeza: {'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length'}


### Geração dos prompts e preparação do dataset para ajuste fino (SFT)

Este bloco cria o conjunto de dados formatado para **Supervised Fine-Tuning (SFT)**, transformando os registros limpos em instruções de treinamento com perguntas e respostas simuladas no estilo de chat.

**Entradas**
- `train`: conjunto de dados limpo.
- `CFG["seed"]`: semente de aleatoriedade para reprodutibilidade.

**Processo (passo a passo)**
1. Define o conjunto de **perguntas padrão** (`QUESTION_TEMPLATES`), que simulam interações típicas de e-commerce, como:
   - “Descreva o produto.”
   - “Liste as principais características.”
   - “Resuma a descrição do item.”
2. Implementa a função `build_prompt(title, question)`:
   - Constrói um prompt no formato **[INST]...[/INST]**, padrão usado por modelos de chat (ex.: Llama, Mistral).
   - Inclui o título do produto e a pergunta, instruindo o modelo a responder apenas com base na descrição.
3. Define `to_sft_rows(rows)`:
   - Para cada item do dataset:
     - Escolhe aleatoriamente uma das perguntas.
     - Gera um prompt completo unindo instrução + conteúdo (`content`).
   - Retorna uma lista de dicionários com a chave `"text"`, contendo os exemplos finais de treino.
4. Gera o conjunto `sft_train_rows` e aplica divisão interna (holdout) para validação:
   - Define a fração de validação (`val_frac`): 5% para datasets grandes, 20% para pequenos.
   - Embaralha e separa os exemplos em treino e validação (`sft_val_rows`).
5. Cria o objeto `DatasetDict` da biblioteca **Hugging Face Datasets** com duas partições:
   - `train`
   - `validation`
6. Exibe o resumo do dataset e um exemplo truncado do primeiro prompt gerado (primeiros 300 caracteres).

**Saídas**
- Objeto `dataset` contendo os conjuntos de treino e validação prontos para o fine-tuning.
- Exemplo de prompt-formatado exibido no console.

**Observações**
- Essa estrutura segue o formato esperado por pipelines de ajuste fino da Hugging Face.
- O uso de prompts em estilo de chat ajuda o modelo a aprender padrões de pergunta e resposta contextualizados.
- A aleatoriedade controlada pela semente (`seed`) garante repetibilidade na geração dos exemplos.


In [7]:
import random
from datasets import Dataset, DatasetDict

random.seed(CFG["seed"])

QUESTION_TEMPLATES = [
    "Descreva o produto.",
    "Liste as principais características.",
    "Resuma a descrição do item.",
]

def build_prompt(title, question):
    return (
        "[INST] Você é um assistente de e-commerce. Responda APENAS com informações da descrição.\n"
        f"Título do produto: {title}\nPergunta: {question} [/INST]"
    )

def to_sft_rows(rows):
    out = []
    for r in rows:
        q = random.choice(QUESTION_TEMPLATES)
        prompt = build_prompt(r["title"], q)
        out.append({"text": prompt + "\n" + r["content"]})
    return out

sft_train_rows = to_sft_rows(train)
# Usaremos o teste como conjunto de avaliação externa; para validação interna criamos um holdout
val_frac = 0.05 if len(sft_train_rows) > 20 else 0.2
val_n = max(1, int(len(sft_train_rows)*val_frac))
random.shuffle(sft_train_rows)
sft_val_rows   = sft_train_rows[:val_n]
sft_train_rows = sft_train_rows[val_n:]

dataset = DatasetDict({
    "train": Dataset.from_list(sft_train_rows) if sft_train_rows else Dataset.from_list([{"text": ""}]),
    "validation": Dataset.from_list(sft_val_rows) if sft_val_rows else Dataset.from_list([{"text": ""}]),
})
print(dataset)
print("Exemplo SFT:", dataset["train"][0]["text"][:300], "...")


c:\Users\PC\Documents\Tech Challenge 3 (1)\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1320597
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 69505
    })
})
Exemplo SFT: [INST] Você é um assistente de e-commerce. Responda APENAS com informações da descrição.
Título do produto: Mighty Peking Man
Pergunta: Resuma a descrição do item. [/INST]
What makesMighty Peking Mansuch a trashy delight? It's not just the absurdly obvious special effects and atrocious dubbing--thos ...


### Verificação detalhada do ambiente PyTorch e da GPU CUDA

Este bloco realiza uma inspeção completa do ambiente de execução do **PyTorch**, verificando compatibilidade com aceleração via **GPU** e suporte a recursos avançados como **BF16 (bfloat16)**. O objetivo é garantir que o hardware e os drivers estão configurados corretamente antes do treinamento.

**Entradas**
- Nenhuma entrada direta. O código apenas consulta o sistema local e as bibliotecas instaladas.

**Processo (passo a passo)**
1. Importa os módulos necessários (`torch`, `importlib`, `platform`).
2. Exibe as versões do **Python** e do **PyTorch** instaladas.
3. Verifica se há uma GPU compatível com **CUDA** disponível (`torch.cuda.is_available()`).
4. Caso exista GPU:
   - Mostra o **nome do dispositivo** detectado.
   - Obtém as **propriedades da GPU** (`torch.cuda.get_device_properties`).
   - Calcula e exibe a **memória total de VRAM** em gigabytes.
   - Checa se o ambiente oferece suporte ao formato **BF16**, útil para treinar modelos de forma mais eficiente em GPUs modernas.
5. Caso não exista GPU, informa que o treinamento ocorrerá apenas em CPU.

**Saídas**
- Informações impressas no console:
  - Versões do Python e PyTorch.
  - Status de disponibilidade da GPU.
  - Nome, VRAM e suporte BF16 (se aplicável).

**Observações**
- Executar este bloco **antes de iniciar o treinamento** ajuda a diagnosticar incompatibilidades com CUDA ou versões incorretas do PyTorch.
- O suporte a **BF16** melhora o desempenho e reduz o consumo de memória em GPUs compatíveis (como A100, H100, RTX 40xx).
- Caso “Sem GPU CUDA detectada.” seja exibido, é necessário verificar se:
  - Os drivers NVIDIA e CUDA Toolkit estão instalados.
  - A versão do PyTorch foi compilada com suporte a CUDA.


In [8]:
import torch, importlib, platform

print("Python:", platform.python_version())
print("PyTorch:", torch.__version__)
print("CUDA disponível:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    props = torch.cuda.get_device_properties(0)
    print("VRAM total (GB):", round(props.total_memory/1024**3, 2))
    bf16_ok = bool(getattr(torch.cuda, "is_bf16_supported", lambda: False)())
    print("BF16 suportado pelo driver/runtime:", bf16_ok)
else:
    print("Sem GPU CUDA detectada.")


Python: 3.11.9
PyTorch: 2.5.1+cu121
CUDA disponível: True
GPU: NVIDIA GeForce RTX 4060 Ti
VRAM total (GB): 8.0
BF16 suportado pelo driver/runtime: True


### Carregamento do modelo com quantização 4-bit + configuração LoRA e SFT (TRL)

Este bloco prepara o pipeline de **fine-tuning supervisionado (SFT)** com economia de memória via **quantização 4-bit (bitsandbytes)** e **adaptação leve LoRA (PEFT)**, deixando o ambiente pronto para iniciar o treinamento.

**Entradas**
- `BASE_MODEL`: nome do modelo base do Hugging Face Hub.
- `CFG`: dicionário de hiperparâmetros (épocas, batch size, seed, etc.).
- `dataset["train"]` e `dataset["validation"]`: datasets formatados com a coluna `"text"`.

**Processo (passo a passo)**
1. **Semente**: fixa `torch.manual_seed(CFG["seed"])` para reprodutibilidade.
2. **Quantização (4-bit)**: cria `BitsAndBytesConfig` com:
   - `load_in_4bit=True` para carregar pesos em 4-bit.
   - `bnb_4bit_use_double_quant=True` e `bnb_4bit_quant_type="nf4"` (quantização NF4 mais estável).
   - `bnb_4bit_compute_dtype=torch.bfloat16` para computação em **BF16** (ganho de performance/estabilidade em GPUs compatíveis).
3. **Tokenizer**:
   - Carrega com `AutoTokenizer.from_pretrained(BASE_MODEL)`.
   - Ajusta `pad_token = eos_token` se o modelo não tiver `pad_token` (evita warnings/erros em batching).
4. **Modelo base quantizado**:
   - `AutoModelForCausalLM.from_pretrained(...)` com `quantization_config=bnb`, `device_map="auto"` (distribui automaticamente nos dispositivos) e `torch_dtype=torch.bfloat16`.
5. **Configuração LoRA** (`LoraConfig`):
   - Usa `r`, `alpha` e `dropout` vindos de `CFG`.
   - Aplica LoRA nos módulos de projeção/MLP típicos de LLMs (`q_proj`, `k_proj`, `v_proj`, `o_proj`, `gate_proj`, `up_proj`, `down_proj`), reduzindo o número de parâmetros treináveis e o consumo de VRAM.
6. **Hiperparâmetros de SFT** (`SFTConfig`):
   - Diretório de saída (`output_dir`), épocas, batch sizes, `gradient_accumulation_steps`.
   - `learning_rate`, `lr_scheduler_type="cosine"`, `warmup_ratio=0.03`, `weight_decay=0.01`.
   - Estratégia de avaliação por passos (`eval_strategy="steps"`, `eval_steps=500`) e checkpoints (`save_steps=500`).
   - `max_steps=1000` e `bf16=True` (usa bfloat16 no treino).
7. **Formatação de exemplos**:
   - `formatting_func` retorna `example["text"]` (cada linha já vem pronta para o modelo).
8. **Instancia o treinador** (`SFTTrainer`):
   - Recebe `model`, `peft_config` (LoRA), `args` (SFTConfig) e os datasets de treino/validação.
9. Imprime confirmação: “Pronto para treinar.”

**Saídas**
- Objeto `trainer` pronto para `trainer.train()`.
- Tokenizer configurado (com `pad_token` definido).
- Modelo base carregado com pesos em 4-bit e cabeçalhos LoRA aplicados.

**Observações**
- **Requisitos**: `bitsandbytes` instalado e GPU com suporte adequado para 4-bit/BF16 (CPU pode não suportar essa configuração).
- **Memória**: quantização 4-bit reduz significativamente o uso de VRAM, permitindo treinar modelos maiores em GPUs comuns.
- **Pad Token**: usar `eos_token` como `pad_token` é prático, mas garanta que métricas e perda não sejam enviesadas por padding (atenção às máscaras de atenção na etapa de treino).
- **Avaliação**: com `eval_strategy="steps"`, a validação ocorre periodicamente; ajuste `eval_steps` e `save_steps` conforme o tamanho do dataset.


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

torch.manual_seed(CFG["seed"])

# Quantização 4-bit (requer bitsandbytes)
bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

lora = LoraConfig(
    r=CFG["lora_r"],
    lora_alpha=CFG["lora_alpha"],
    lora_dropout=CFG["lora_dropout"],
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

sft_args = SFTConfig(
    output_dir=str(OUT_DIR),
    num_train_epochs=CFG["num_train_epochs"],
    per_device_train_batch_size=CFG["per_device_train_batch_size"],
    per_device_eval_batch_size=CFG["per_device_eval_batch_size"],
    gradient_accumulation_steps=CFG["grad_acc_steps"],
    learning_rate=CFG["learning_rate"],
    logging_steps=50,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    max_steps=1000,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,
    bf16=True,
    seed=CFG["seed"],
)

def formatting_func(example):
    return example["text"]

trainer = SFTTrainer(
    model=model,
    peft_config=lora,
    args=sft_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    formatting_func=formatting_func,
)

print("Pronto para treinar.")



Truncating eval dataset: 100%|██████████| 69505/69505 [00:00<00:00, 1418510.08 examples/s]

Pronto para treinar.


### Execução do treinamento e salvamento do modelo final

Este bloco realiza a etapa final do pipeline: **treinar o modelo ajustado** e **salvar os resultados** para uso posterior ou implantação.

**Entradas**
- Objeto `trainer` configurado anteriormente, que contém:
  - O modelo base com LoRA e quantização aplicadas.
  - Os datasets de treino e validação.
  - Os parâmetros de otimização definidos em `SFTConfig`.

**Processo (passo a passo)**
1. **Treinamento supervisionado (`trainer.train()`)**  
   Inicia o processo de ajuste fino (fine-tuning) do modelo, com base no dataset preparado.  
   Durante a execução:
   - O modelo é atualizado apenas nas camadas LoRA (parametrização leve).
   - São exibidas métricas de perda (loss) e, se configuradas, métricas de avaliação periódicas.
   - Checkpoints intermediários são salvos conforme definido em `save_steps`.
2. **Salvamento do modelo ajustado (`trainer.save_model`)**  
   Exporta os pesos resultantes do treinamento para o diretório `OUT_DIR/final`.  
   Este passo garante que as modificações feitas durante o fine-tuning possam ser recarregadas posteriormente.
3. **Salvamento do tokenizer (`tokenizer.save_pretrained`)**  
   Salva o tokenizador no mesmo diretório, garantindo compatibilidade com o modelo durante a inferência.

**Saídas**
- Diretório final `models/out-sft/final/` contendo:
  - Arquivos do modelo ajustado (pesos LoRA e configurações).
  - Tokenizador (`tokenizer.json`, `special_tokens_map.json`, etc.).
- Log de progresso e métricas do treinamento exibidos no console.

**Observações**
- Após essa etapa, o modelo está pronto para **inferência ou avaliação**, podendo ser carregado via `from_pretrained("models/out-sft/final")`.
- O tempo de execução depende do tamanho do dataset, das configurações de batch size e da GPU utilizada.
- É boa prática verificar a **perda final (loss)** e testar o modelo em exemplos reais antes de distribuí-lo.


In [13]:
trainer.train()
trainer.save_model(str(OUT_DIR / "final"))
tokenizer.save_pretrained(str(OUT_DIR / "final"))

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
c:\Users\PC\Documents\Tech Challenge 3 (1)\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
500,1.774900,1.702535,1.714422,3797894.000000,0.645449
1000,1.776200,1.692364,1.696128,7551922.000000,0.647121


c:\Users\PC\Documents\Tech Challenge 3 (1)\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


('models\\out-sft\\final\\tokenizer_config.json',
 'models\\out-sft\\final\\special_tokens_map.json',
 'models\\out-sft\\final\\chat_template.jinja',
 'models\\out-sft\\final\\tokenizer.model',
 'models\\out-sft\\final\\added_tokens.json',
 'models\\out-sft\\final\\tokenizer.json')